In [1]:
import gc
import sys
sys.path.append('../')
import time
from collections import namedtuple
from tqdm import tqdm
import numpy as np
import torch
import torchvision
import src.utils as utils
from src.models import LeNetMNIST, ResVestimatorMNIST, R18VestimatorMNIST, Vestimator
from src.frameworks.online_dvrl import Odvrl

In [2]:
from collections import namedtuple
Parameters = namedtuple('Parameters', [
    'saving_path',
    'val_batch_size',
    'epochs',
    'device',
    'learning_rate',
    'num_workers',
    # 'input_dim',
    # 'hidden_dim',
    # 'output_dim',
    # 'layer_number',
])

In [3]:
parameters = Parameters(
    saving_path='../logs',
    val_batch_size=32,
    epochs=150,
    device='cuda',
    learning_rate=1e-4,
    num_workers=1,
    # input_dim=794,
    # hidden_dim=100,
    # output_dim=10,
    # layer_number=5,
)

In [4]:
T = 10
noise_level = 0.2
seed = 3407
num_weak = 30
pred_model = LeNetMNIST()
val_model = LeNetMNIST()
value_estimator = R18VestimatorMNIST()

In [5]:
np.random.seed(seed)
torch.manual_seed(seed)

In [6]:
# run 'pretrain_mnist.ipynb' first
# then load this pretrained model
state_dict = torch.load("../data/pretrained_models/pretrained_lenetmnist.pt")
val_model.load_state_dict(state_dict)

<All keys matched successfully>

In [7]:
x_train, y_train, x_test, y_test, noisy_idxs = utils.create_noisy_mnist(method='uniform', noise_level=noise_level)

In [8]:
noisy_idxs.sort()
noisy_idxs[:20]

array([ 6,  6,  9, 23, 28, 34, 36, 41, 52, 53, 57, 63, 66, 66, 68, 74, 82,
       82, 82, 86])

In [9]:
x_train = torch.tensor(x_train)
y_train = torch.tensor(y_train)
# test_data = torch.utils.data.TensorDataset(torch.tensor(x_test), torch.tensor(y_test))
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.1307,), (0.3081,))
])
test_data = torchvision.datasets.MNIST('../data', train=False, download=True, transform=transform)

In [10]:
num_data = len(y_train)

In [11]:
engine = Odvrl(num_weak=num_weak, pred_model=pred_model, val_model=val_model, value_estimator=value_estimator, parameters=parameters)

In [12]:
time.asctime()

'Fri May  5 16:00:59 2023'

In [13]:
subset_len = num_data // T
for t in range(T):
    start_id = t * subset_len
    end_id = min((t + 1) * subset_len, num_data)
    engine.one_step(
        t, 
        X=x_train[start_id:end_id], 
        y=y_train[start_id:end_id], 
        val_dataset=test_data, 
    )
    utils.super_save()
    current_noisy_idxs = np.extract((noisy_idxs >= start_id) & (noisy_idxs < end_id), noisy_idxs)
    current_corrupted_num = len(current_noisy_idxs)
    if current_corrupted_num > 0:
        values = []
        for i in range(subset_len // 32):
            part_values = engine.evaluate(x_train[start_id+i*32: start_id+min((i+1)*32, num_data)], y_train[start_id+i*32: start_id+min((i+1)*32, num_data)])
            values = np.concatenate((values, part_values))
            utils.super_save()
        guess_idxs = np.argsort(values)
        print(guess_idxs[:10])
        print(current_noisy_idxs[:10])
        discover_rate = len(np.intersect1d(start_id+guess_idxs[:current_corrupted_num], current_noisy_idxs)) / current_corrupted_num
        print('discover rate: {}'.format(discover_rate))

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 6.00 GiB total capacity; 5.26 GiB already allocated; 0 bytes free; 5.29 GiB reserved in total by PyTorch)

In [ ]:
time.asctime()